In [1]:
import pandas as pd
import requests
import json
import gmaps
from pprint import pprint

# Google developer API key
from config import gkey


In [2]:
#import csv file as dataframe
redfin_closer_df = pd.read_csv('redfin_2020-05-31-12-00-06.csv')
redfin_closer_df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Daly-City/246-Monte-V...,MLSListings,ML81794298,Y,Y,37.658315,-122.463216
1,MLS Listing,NaN,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Leandro/1950-Ever...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40905479,N,Y,37.722627,-122.136859
2,MLS Listing,NaN,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40904884,N,Y,37.741184,-122.178730
3,MLS Listing,NaN,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/2543-Lancaste...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906691,N,Y,37.664743,-122.051583
4,MLS Listing,NaN,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906683,N,Y,37.797502,-122.241082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,New Construction Plan,NaN,Single Family Residential,Stellar 820,San Francisco,CA,94123,254390,2,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,NewHomeSource.com,Plan-121706983,N,Y,37.798864,-122.437717
173,New Construction Plan,NaN,Single Family Residential,Residence 3 WLH,Richmond,CA,94804,595000,3,3.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,NewHomeSource.com,Plan-121781294,N,Y,37.920715,-122.355477
174,New Construction Plan,NaN,Single Family Residential,Plan 5 WLH,Hayward,CA,94544,800000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,NewHomeSource.com,Plan-121781292,N,Y,37.632475,-122.049066
175,New Construction Plan,NaN,Single Family Residential,Plan 3 Modeled,Hayward,CA,94544,769000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,NewHomeSource.com,Plan-111707850,N,Y,37.647801,-122.064880


In [3]:
#rename URL column for readability
redfin_closer_url_cleaned = redfin_closer_df.rename(columns={"URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)":"URL"})


In [4]:
redfin_closer_url_cleaned

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Daly-City/246-Monte-V...,MLSListings,ML81794298,Y,Y,37.658315,-122.463216
1,MLS Listing,NaN,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Leandro/1950-Ever...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40905479,N,Y,37.722627,-122.136859
2,MLS Listing,NaN,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40904884,N,Y,37.741184,-122.178730
3,MLS Listing,NaN,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/2543-Lancaste...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906691,N,Y,37.664743,-122.051583
4,MLS Listing,NaN,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906683,N,Y,37.797502,-122.241082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,New Construction Plan,NaN,Single Family Residential,Stellar 820,San Francisco,CA,94123,254390,2,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,NewHomeSource.com,Plan-121706983,N,Y,37.798864,-122.437717
173,New Construction Plan,NaN,Single Family Residential,Residence 3 WLH,Richmond,CA,94804,595000,3,3.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,NewHomeSource.com,Plan-121781294,N,Y,37.920715,-122.355477
174,New Construction Plan,NaN,Single Family Residential,Plan 5 WLH,Hayward,CA,94544,800000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,NewHomeSource.com,Plan-121781292,N,Y,37.632475,-122.049066
175,New Construction Plan,NaN,Single Family Residential,Plan 3 Modeled,Hayward,CA,94544,769000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,NewHomeSource.com,Plan-111707850,N,Y,37.647801,-122.064880


In [ ]:
#Pulling out only necessary columns for analysis

# locations2 = pd.DataFrame(redfin_closer_df2[["ADDRESS", "CITY", "URL", "LATITUDE", "LONGITUDE"]])
# locations2


In [5]:
#Add column with lat and long combined 

redfin_closer_url_cleaned["LOCATION"] = redfin_closer_url_cleaned["LATITUDE"].astype(str) +"," + redfin_closer_url_cleaned["LONGITUDE"].astype(str)

#Add columns to add API data into
redfin_closer_url_cleaned["travel_time"] = ""
redfin_closer_url_cleaned["travel_cost"] = ""
redfin_closer_url_cleaned["travel_seconds"] = ""



In [6]:
redfin_closer_url_cleaned.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,travel_time,travel_cost,travel_seconds
0,MLS Listing,NaN,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,...,http://www.redfin.com/CA/Daly-City/246-Monte-V...,MLSListings,ML81794298,Y,Y,37.658315,-122.463216,,,
1,MLS Listing,NaN,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,...,http://www.redfin.com/CA/San-Leandro/1950-Ever...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40905479,N,Y,37.722627,-122.136859,,,
2,MLS Listing,NaN,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,...,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40904884,N,Y,37.741184,-122.178730,,,
3,MLS Listing,NaN,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,...,http://www.redfin.com/CA/Hayward/2543-Lancaste...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906691,N,Y,37.664743,-122.051583,,,
4,MLS Listing,NaN,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,...,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906683,N,Y,37.797502,-122.241082,,,


In [8]:
outputFormat = "json"
# outputFormat = "xml"

params = {
#     "destination": "37.29014229999999,-121.8443185",
    "origin": "37.776501,-122.417018",
    "mode": "transit"
}

base_url = f"https://maps.googleapis.com/maps/api/directions/{outputFormat}?{params}&key={gkey}"


# use iterrows to iterate through pandas dataframe
for index, row in redfin_closer_url_cleaned.iterrows():

    # get restaurant type from df
    house_loc = row['LOCATION']

    # add keyword to params dict
    params['destination'] = house_loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {house_loc}.")
    response = requests.get(base_url, params=params).json()


# pprint(response)

    # extract results
    try:
        fare_results = response['routes'][0]['fare']['text']
        redfin_closer_url_cleaned.loc[index, 'travel_cost'] = fare_results
        print(f"Cost for travel is: {fare_results}")
    except (KeyError, IndexError):
        print("Missing travel_cost... skipping.")
        print("------------")

    try:
        time_results = response['routes'][0]['legs'][0]['duration']['text']
        seconds_results = response['routes'][0]['legs'][0]['duration']['value']
        redfin_closer_url_cleaned.loc[index, 'travel_time'] = time_results
        redfin_closer_url_cleaned.loc[index, 'travel_seconds'] = seconds_results

        print(f"Travel time from location is: {time_results}")
        print("------------")
    except (KeyError, IndexError):
        print("Missing travel_time... skipping.")
        print("------------")

Retrieving Results for Index 0: 37.658315,-122.46321599999999.
Cost for travel is: $6.65
Travel time from location is: 1 hour 10 mins
------------
Retrieving Results for Index 1: 37.722626500000004,-122.13685929999998.
Missing travel_cost... skipping.
------------
Travel time from location is: 58 mins
------------
Retrieving Results for Index 2: 37.74118370000001,-122.1787301.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 13 mins
------------
Retrieving Results for Index 3: 37.6647434,-122.0515834.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 32 mins
------------
Retrieving Results for Index 4: 37.7975023,-122.2410824.
Missing travel_cost... skipping.
------------
Travel time from location is: 51 mins
------------
Retrieving Results for Index 5: 37.9338641,-122.08545330000001.
Cost for travel is: $9.30
Travel time from location is: 9 hours 40 mins
------------
Retrieving Results for Index 6: 37.758351,-122.19342

Cost for travel is: $3.00
Travel time from location is: 40 mins
------------
Retrieving Results for Index 52: 37.681411499999996,-122.12062140000002.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 3 mins
------------
Retrieving Results for Index 53: 37.7560687,-122.1527176.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 22 mins
------------
Retrieving Results for Index 54: 37.6223473,-122.40884229999999.
Missing travel_cost... skipping.
------------
Travel time from location is: 51 mins
------------
Retrieving Results for Index 55: 37.8041754,-122.19271909999999.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 10 mins
------------
Retrieving Results for Index 56: 37.764511,-122.1589273.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 30 mins
------------
Retrieving Results for Index 57: 37.792353000000006,-122.22661699999999.
Missing travel_cost.

Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 58 mins
------------
Retrieving Results for Index 104: 37.9087019,-122.3459787.
Cost for travel is: $6.65
Travel time from location is: 1 hour 36 mins
------------
Retrieving Results for Index 105: 37.8310122,-122.27413940000001.
Cost for travel is: $4.45
Travel time from location is: 36 mins
------------
Retrieving Results for Index 106: 37.7851394,-122.18571329999999.
Missing travel_cost... skipping.
------------
Travel time from location is: 57 mins
------------
Retrieving Results for Index 107: 37.7268617,-122.4279784.
Cost for travel is: $3.00
Travel time from location is: 30 mins
------------
Retrieving Results for Index 108: 37.8227361,-122.28082409999999.
Missing travel_cost... skipping.
------------
Travel time from location is: 37 mins
------------
Retrieving Results for Index 109: 37.708813899999996,-122.12734309999999.
Missing travel_cost... skipping.
------------
Travel time from location is

Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 27 mins
------------
Retrieving Results for Index 155: 37.7284085,-122.1720107.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 5 mins
------------
Retrieving Results for Index 156: 37.7672261,-122.1659348.
Missing travel_cost... skipping.
------------
Travel time from location is: 1 hour 22 mins
------------
Retrieving Results for Index 157: 37.9260938,-122.3283113.
Cost for travel is: $9.80
Travel time from location is: 53 mins
------------
Retrieving Results for Index 158: 37.744493799999994,-122.1502669.
Missing travel_cost... skipping.
------------
Travel time from location is: 57 mins
------------
Retrieving Results for Index 159: 37.7857576,-122.23146470000002.
Missing travel_cost... skipping.
------------
Travel time from location is: 51 mins
------------
Retrieving Results for Index 160: 37.669989,-122.07225700000001.
Missing travel_cost... skipping.
----------

In [9]:
redfin_closer_url_cleaned

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,travel_time,travel_cost,travel_seconds
0,MLS Listing,NaN,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,...,http://www.redfin.com/CA/Daly-City/246-Monte-V...,MLSListings,ML81794298,Y,Y,37.658315,-122.463216,1 hour 10 mins,$6.65,4212
1,MLS Listing,NaN,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,...,http://www.redfin.com/CA/San-Leandro/1950-Ever...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40905479,N,Y,37.722627,-122.136859,58 mins,,3476
2,MLS Listing,NaN,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,...,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40904884,N,Y,37.741184,-122.178730,1 hour 13 mins,,4401
3,MLS Listing,NaN,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,...,http://www.redfin.com/CA/Hayward/2543-Lancaste...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906691,N,Y,37.664743,-122.051583,1 hour 32 mins,,5497
4,MLS Listing,NaN,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,...,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906683,N,Y,37.797502,-122.241082,51 mins,,3087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,New Construction Plan,NaN,Single Family Residential,Stellar 820,San Francisco,CA,94123,254390,2,2.0,...,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,NewHomeSource.com,Plan-121706983,N,Y,37.798864,-122.437717,17 mins,$4.50,1040
173,New Construction Plan,NaN,Single Family Residential,Residence 3 WLH,Richmond,CA,94804,595000,3,3.0,...,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,NewHomeSource.com,Plan-121781294,N,Y,37.920715,-122.355477,1 hour 5 mins,$9.80,3897
174,New Construction Plan,NaN,Single Family Residential,Plan 5 WLH,Hayward,CA,94544,800000,4,3.5,...,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,NewHomeSource.com,Plan-121781292,N,Y,37.632475,-122.049066,1 hour 20 mins,,4782
175,New Construction Plan,NaN,Single Family Residential,Plan 3 Modeled,Hayward,CA,94544,769000,4,3.5,...,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,NewHomeSource.com,Plan-111707850,N,Y,37.647801,-122.064880,1 hour 24 mins,,5039


In [10]:
#converting seconds to minutes for usability
redfin_closer_url_cleaned['travel_minutes'] = (pd.to_numeric(redfin_closer_url_cleaned['travel_seconds']) / 60)


In [12]:
# redfin_closer_url_cleaned



In [13]:
list(redfin_closer_url_cleaned.columns)

['SALE TYPE',
 'SOLD DATE',
 'PROPERTY TYPE',
 'ADDRESS',
 'CITY',
 'STATE OR PROVINCE',
 'ZIP OR POSTAL CODE',
 'PRICE',
 'BEDS',
 'BATHS',
 'LOCATION',
 'SQUARE FEET',
 'LOT SIZE',
 'YEAR BUILT',
 'DAYS ON MARKET',
 '$/SQUARE FEET',
 'HOA/MONTH',
 'STATUS',
 'NEXT OPEN HOUSE START TIME',
 'NEXT OPEN HOUSE END TIME',
 'URL',
 'SOURCE',
 'MLS#',
 'FAVORITE',
 'INTERESTED',
 'LATITUDE',
 'LONGITUDE',
 'travel_time',
 'travel_cost',
 'travel_seconds',
 'travel_minutes']

In [14]:
#drop unneeded columns
redfin_closer_columns_dropped = redfin_closer_url_cleaned.drop(columns=['SALE TYPE', 'SOLD DATE', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'SOURCE'])



In [15]:
redfin_closer_columns_dropped

,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,...,URL,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,travel_time,travel_cost,travel_seconds,travel_minutes
0,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,"37.658315,-122.46321599999999",1600.0,...,http://www.redfin.com/CA/Daly-City/246-Monte-V...,ML81794298,Y,Y,37.658315,-122.463216,1 hour 10 mins,$6.65,4212,70.200000
1,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,"37.722626500000004,-122.13685929999998",1940.0,...,http://www.redfin.com/CA/San-Leandro/1950-Ever...,40905479,N,Y,37.722627,-122.136859,58 mins,,3476,57.933333
2,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,"37.74118370000001,-122.1787301",1414.0,...,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,40904884,N,Y,37.741184,-122.178730,1 hour 13 mins,,4401,73.350000
3,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,"37.6647434,-122.0515834",1623.0,...,http://www.redfin.com/CA/Hayward/2543-Lancaste...,40906691,N,Y,37.664743,-122.051583,1 hour 32 mins,,5497,91.616667
4,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,"37.7975023,-122.2410824",1450.0,...,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,40906683,N,Y,37.797502,-122.241082,51 mins,,3087,51.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Single Family Residential,Stellar 820,San Francisco,CA,94123,254390,2,2.0,"37.798864,-122.43771699999999",820.0,...,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,Plan-121706983,N,Y,37.798864,-122.437717,17 mins,$4.50,1040,17.333333
173,Single Family Residential,Residence 3 WLH,Richmond,CA,94804,595000,3,3.0,"37.920715,-122.35547700000001",1709.0,...,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,Plan-121781294,N,Y,37.920715,-122.355477,1 hour 5 mins,$9.80,3897,64.950000
174,Single Family Residential,Plan 5 WLH,Hayward,CA,94544,800000,4,3.5,"37.632475,-122.049066",2001.0,...,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,Plan-121781292,N,Y,37.632475,-122.049066,1 hour 20 mins,,4782,79.700000
175,Single Family Residential,Plan 3 Modeled,Hayward,CA,94544,769000,4,3.5,"37.6478013,-122.06487959999998",2074.0,...,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,Plan-111707850,N,Y,37.647801,-122.064880,1 hour 24 mins,,5039,83.983333


In [16]:
redfin_closer_columns_dropped.to_csv("redfin_output_csv_price")
